# Aim

- Extract the list of mitochondria within astroyctes with a labelled soma astro_mito.csv
- Label each mitochondria as being either in a branch or soma close by using the distance to the soma position astro_mito_extrastats.csc

## How does an astroycte look like?

Unfortunately, the astryoctes seem to absent in the pinky sandbox version of the dataset which is easy to get via navis.

In [26]:
import navis.interfaces.microns as mi

client = mi.get_cave_client(datastack="pinky_sandbox")

client.materialize.get_tables()

glia = client.materialize.query_table("glia")

glia.cell_type.value_counts()

microglia    26
OPC          18
Name: cell_type, dtype: int64

The astrocytes can however be visualized and most probably also accessed via the published version v185.4, available on 

https://layer23.microns-explorer.org/

To select an astroycte look up its seg id in the cell dataframe.

In [27]:
import pandas 

In [28]:
cell_df = pandas.read_csv("smo_data/cell_type_classification.csv", index_col=0)

In [29]:
cell_df.query("cell_segid==648518346349539853")

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
757,96864,51376,820,648518346349539853,pyramidal,pyramidal cell


In [30]:
cell_df.query("cell_subtype=='astrocyte'").head(3)

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte


A quick [look](https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22segments%22:%5B%22648518346341392909%22%2C%22648518346349517321%22%2C%22648518346349538235%22%5D%2C%22hiddenSegments%22:%5B%22648518346349539462%22%2C%22648518346349539853%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%2C%7B%22source%22:%22precomputed://https://td.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B87587.578125%2C56984.7890625%2C825.427734375%5D%7D%7D%2C%22zoomFactor%22:27.009461089941993%7D%2C%22perspectiveOrientation%22:%5B-0.2821060121059418%2C-0.1709754317998886%2C0.40289685130119324%2C-0.853731632232666%5D%2C%22perspectiveZoom%22:1089.9513719139259%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22cell_segmentation_v185%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D) shows that astrocytes might be non-uniform in space, so one needs some kind of spatial position of the soma to really do this form of simple spatial filteriung.

## Get soma positions of astrocytes

One could check whether the same astrocytes are also availabe in the soma valence table, which might provide a soma position.

In [31]:
! wget -O smo_data/soma.csv https://zenodo.org/records/7510511/files/soma_valence_v185.csv?download=1

--2025-05-15 09:56:02--  https://zenodo.org/records/7510511/files/soma_valence_v185.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33983 (33K) [text/plain]
Saving to: ‘smo_data/soma.csv’

smo_data/soma.csv   100%[===================>]  33,19K  62,1KB/s    in 0,5s    

2025-05-15 09:56:03 (62,1 KB/s) - ‘smo_data/soma.csv’ saved [33983/33983]



In [32]:
soma_df = pandas.read_csv("smo_data/soma.csv")

In [33]:
soma_df.query("cell_type=='g'").head(2)

,Unnamed: 0,id,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,nuc_id
162,162,365,g,[61834 40655 1322],648518346349527319,247336,162620,52880,4308
210,210,364,g,[91882 59312 1521],648518346347160236,367528,237248,60840,7987


In [34]:
cws_df = soma_df.merge(right=cell_df, right_on="cell_segid",left_on="pt_root_id")

In [35]:
cws_df.cell_subtype.value_counts().astrocyte

16

In [36]:
cws_df.query("cell_subtype=='astrocyte'")

,Unnamed: 0,id,cell_type_x,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,nuc_id,centroid_x,centroid_y,centroid_z,cell_segid,cell_type_y,cell_subtype
162,162,365,g,[61834 40655 1322],648518346349527319,247336,162620,52880,4308,61616,40400,1342,648518346349527319,glia,astrocyte
211,214,369,g,[63741 44570 1069],648518346349517141,254964,178280,42760,4766,63568,44352,1066,648518346349517141,glia,astrocyte
214,219,374,g,[101058 49862 53],648518346349386860,404232,199448,2120,5709,101120,49760,60,648518346349386860,glia,astrocyte
215,220,375,g,[110817 60682 237],648518346342795947,443268,242728,9480,8656,110816,60768,234,648518346342795947,glia,astrocyte
221,228,411,g,[69091 74601 1902],648518346349527316,276364,298404,76080,13913,69184,74448,1889,648518346349527316,glia,astrocyte
224,235,392,g,[51194 62310 354],648518346349528249,204776,249240,14160,8583,51200,62384,363,648518346349528249,glia,astrocyte
226,237,423,g,[56945 55996 815],648518346349528250,227780,223984,32600,6542,56976,56032,785,648518346349528250,glia,astrocyte
229,240,391,g,[50598 67772 997],648518346349521344,202392,271088,39880,9655,50256,67696,990,648518346349521344,glia,astrocyte
231,243,393,g,[53383 62029 354],648518346349536487,213532,248116,14160,8589,53136,62176,320,648518346349536487,glia,astrocyte
234,250,402,g,[87040 59386 98],648518346349498574,348160,237544,3920,7533,87104,59520,93,648518346349498574,glia,astrocyte


One way to create the mask is then to get the nucleus volume, add 5um to all directions to get the soma and filter mitochondria out that are within this volume...

Note: For astroycytes without a detected nucleus in the volume, one can argue that the mitos are all within astrocyte branches.

Note: In the above table there is one astroycte with segid 648518346349525715 that has two associated nuclei, so probably a merge of two astrocytes. I will therefore remove this astroycte from the analysed cells.

In [37]:
mito_df = pandas.read_csv("smo_data/mito.csv")

In [38]:
astro_with_soma_ids = cws_df.query("cell_subtype=='astrocyte'").drop_duplicates(subset=["cell_segid"], keep=False)["cell_segid"].values

astro_mitos = mito_df.loc[mito_df["cellid"].isin(astro_with_soma_ids) ]

In [39]:
len(astro_with_soma_ids)

14

In [40]:
len(astro_mitos)

10577

In [41]:
astro_mitos

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
19,809674,1280360,57366,67234,44,56430,66442,21,58286,67920,66,648518346349536487,205370.28,240697.72,1760
39,1377605,932952,66374,70988,1695,66154,70590,1667,66740,71544,1746,648518346349527316,237618.92,254137.04,67800
43,925473,54528,58750,68504,1964,58708,68398,1960,58800,68606,1970,648518346349530569,210325.00,245244.32,78560
56,3280601,266076,101786,51008,36,101626,50760,25,101932,51150,52,648518346349386860,364393.88,182608.64,1440
77,920455,257268,58702,67270,1422,58582,67174,1412,58796,67348,1433,648518346349530569,210153.16,240826.60,56880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409561,1307867,2159164,66136,41558,511,65316,40844,465,66644,41964,545,648518346349517141,236766.88,148777.64,20440
2409562,1039174,2184080,59536,69916,2055,58758,69474,1983,60464,70546,2157,648518346349530569,213138.88,250299.28,82200
2409565,983093,2220768,59472,45154,453,59236,44948,346,59906,45408,562,648518346349517141,212909.76,161651.32,18120
2409567,2163926,2238244,80264,58306,380,79636,57416,349,80842,58974,401,648518346349498574,287345.12,208735.48,15200


In [42]:
astro_mitos.set_index("mito_id").to_csv("smo_data/astro_mito.csv")